# Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn



In [1]:
import pandas as pd

In [2]:
checkins = pd.read_csv("checkins.dat", low_memory=False, sep='|', skipinitialspace = True) 
checkins.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,---------+---------+----------+---------------...,NaN,NaN,NaN,NaN,NaN
1,984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
3,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43


In [3]:
checkins.rename(columns=str.strip, inplace=True) # в оригинальном датасете в именах колонок куча пробелов. Уберем для удобства
X=checkins.loc[:,("latitude", "longitude")]
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021968 entries, 0 to 1021967
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   latitude   396634 non-null  float64
 1   longitude  396634 non-null  float64
dtypes: float64(2)
memory usage: 15.6 MB


Видно, что для существенного числа наблюдений значение не определены. Отбросим их за ненадобностью.

In [4]:
X.dropna(inplace=True)
X.shape

(396634, 2)

Осталось довольно много точек. Перейдем непосредственно к кластеризации.

In [5]:
from sklearn.cluster import MeanShift
import numpy as np
clustering = MeanShift(bandwidth=0.1).fit(X)

In [20]:
centers=clustering.cluster_centers_
centers.shape

(5536, 2)

Удалим кластеры, содержащие менее 15 точек.

In [21]:
cluster_sizes = np.bincount(clustering.labels_)
centers  = centers[cluster_sizes > 15]
centers.shape

(1304, 2)

Зафиксируем координаты  офисов.

In [8]:

ofices =[
    ("Los Angeles", (33.751277, -118.188740)),
    ("Miami", (25.867736, -80.324116)),
    ("London", (51.503016, -0.075479)),
    ("Amsterdam", (52.378894, 4.885084)),
    ("Beijing", (39.366487, 117.036146)),
    ("Sydney", (-33.868457, 151.205134))
]
ofices

[('Los Angeles', (33.751277, -118.18874)),
 ('Miami', (25.867736, -80.324116)),
 ('London', (51.503016, -0.075479)),
 ('Amsterdam', (52.378894, 4.885084)),
 ('Beijing', (39.366487, 117.036146)),
 ('Sydney', (-33.868457, 151.205134))]

In [9]:
from sklearn.metrics.pairwise import euclidean_distances

Вычислим расстояние от каждого из центров до каждого офиса.

In [10]:
dist = euclidean_distances(centers, [x[1] for x in ofices])

Выведем список 20 ближайших к офисам центров кластеров с указанием соответствующих офисов.

In [64]:
top20 = np.unravel_index(dist.argsort(axis=None)[:20], dist.shape)
top20 = list(zip(top20[0], top20[1]))

for i, pair in enumerate(top20):
    city = ofices[pair[1]][0]
    distance = dist[pair[0]][pair[1]]
    center_coords = centers[pair[0]]
    cluster_size = cluster_sizes[pair[0]]
    print(f'#{i + 1}: {city} at {center_coords.round(2)} (distance={round(distance, 3)}, cluster size = {cluster_size})')

#1: Sydney at [-33.87 151.21] (distance=0.003, cluster size = 215)
#2: Amsterdam at [52.37  4.89] (distance=0.01, cluster size = 159)
#3: Miami at [ 25.85 -80.31] (distance=0.025, cluster size = 165)
#4: London at [51.5  -0.13] (distance=0.052, cluster size = 1185)
#5: Los Angeles at [  33.81 -118.14] (distance=0.075, cluster size = 1258)
#6: Miami at [ 25.79 -80.22] (distance=0.136, cluster size = 2769)
#7: Miami at [ 26.01 -80.21] (distance=0.181, cluster size = 578)
#8: Los Angeles at [  33.9  -118.06] (distance=0.194, cluster size = 614)
#9: London at [51.48 -0.31] (distance=0.234, cluster size = 36)
#10: London at [51.6  -0.32] (distance=0.264, cluster size = 28)
#11: Miami at [ 26.14 -80.34] (distance=0.272, cluster size = 99)
#12: Los Angeles at [  33.92 -118.41] (distance=0.283, cluster size = 1626)
#13: Miami at [ 26.11 -80.16] (distance=0.292, cluster size = 834)
#14: Los Angeles at [  33.82 -117.89] (distance=0.302, cluster size = 1834)
#15: Los Angeles at [  34.06 -118.27] 

Видно, что некоторые центры кластеризации расположенны довольно близко друг к другу. Возможно, имеет смысл схлапывать некоторые из них. С другой стороны, это могут быть, например, крупные торговые центры. Тогда в каждом из них может быть резонно разместить баннер, поскольку это обеспечит большой охват.
Тем не менее, требуемый центр с минимальнымрасстоянием определяется довольно о

In [65]:
top_point = centers[top20[0][0]]

In [59]:
with open("answer_clasterization.txt", "w+") as file:
    file.write(f'{top_point[0]} {top_point[1]}')